# Comparing Square (barcodes) to Dear (SKUs) + more

In [1]:
import pandas as pd
#import matplotlib.pyplot as plt
#import matplotlib.dates as mdates
import numpy as np
#from datetime import datetime
#from datetime import date

## Get data

In [2]:
#DEAR -> PRODUCTS -> EXPORT -> Inventory (without descriptions)

dear = pd.read_csv('inputs/InventoryList_2022-09-06.csv')
dear.head(2)

,ProductCode,Name,Category,Brand,Type,FixedAssetType,CostingMethod,Length,Width,Height,...,DimensionAttribute1,DimensionAttribute2,DimensionAttribute3,DimensionAttribute4,DimensionAttribute5,DimensionAttribute6,DimensionAttribute7,DimensionAttribute8,DimensionAttribute9,DimensionAttribute10
0,NaN,NaN,Service,NaN,Service,NaN,FIFO,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,777823111113,1 Water 355ml,Non Alcoholic Beverage,NaN,Stock,NaN,FIFO,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
# https://squareup.com/dashboard/items/library Export All Items, going with excel format here, easy change for csv

square = pd.read_csv('inputs/catalog-2022-09-06-0411.csv') #,skiprows=5
square.head(2)

,Token,Item Name,Variation Name,SKU,Description,Category,Price,Option Name 1,Option Value 1,Enabled 1,...,Stock Alert Count 3,Price 3,Enabled Warehouse,Current Quantity Warehouse,New Quantity Warehouse,Stock Alert Enabled Warehouse,Stock Alert Count Warehouse,Price Warehouse,Tax - Sales Tax (8.25%),Tax - Sales Tax 2 (8.25%)
0,SF3IB2F2DKQRJCWBV4WJB6N4,1 Water 355ml,Regular,777823111113,NaN,Non Alcoholic Beverage,1.50,NaN,NaN,Y,...,NaN,NaN,Y,0.0,NaN,NaN,NaN,NaN,Y,Y
1,RXYY264UGDD6LEIAZMPXPRPN,11 Below Brewing Taco Tuesday Cans 6 Pack,Regular,851233007157,NaN,Craft Beer,12.99,NaN,NaN,Y,...,NaN,NaN,Y,0.0,NaN,NaN,NaN,NaN,Y,Y


## Get rid of excessive columns, prep names for join

In [4]:
#dear.columns
#square.columns

In [5]:
dear = dear[['ProductCode','Name','Category','Barcode','PriceTier1','Status']]
square = square[['Item Name','SKU','Category','Price','Enabled 1','Enabled 2','Enabled 3','Enabled Warehouse','Tax - Sales Tax (8.25%)', 'Tax - Sales Tax 2 (8.25%)']]

In [6]:
dear = dear.rename(columns={'PriceTier1':'Price'}) #,'ProductCode':'SKU'
dear.head(3)

,ProductCode,Name,Category,Barcode,Price,Status
0,NaN,NaN,Service,NaN,50.00,ACTIVE
1,777823111113,1 Water 355ml,Non Alcoholic Beverage,777823111113,1.50,ACTIVE
2,851233007157,11 Below Brewing Taco Tuesday Cans 6 Pack,Craft Beer,851233007157,12.99,ACTIVE


In [7]:
square = square.rename(columns={'Item Name':'Name','Tax - Sales Tax (8.25%)':'Tax 1','Tax - Sales Tax 2 (8.25%)':'Tax 2'})#renaming for legibility
square.head(3)

,Name,SKU,Category,Price,Enabled 1,Enabled 2,Enabled 3,Enabled Warehouse,Tax 1,Tax 2
0,1 Water 355ml,777823111113,Non Alcoholic Beverage,1.50,Y,Y,Y,Y,Y,Y
1,11 Below Brewing Taco Tuesday Cans 6 Pack,851233007157,Craft Beer,12.99,Y,Y,Y,Y,Y,Y
2,12/24 Mixing Tin Sets,1234500000000,Bar Tools,22.00,Y,Y,Y,Y,Y,Y


## Let's start verifying Dear items

In [8]:
#dear[dear.ProductCode != dear.Barcode]#.to_csv('outputs/badbarcodes.csv')

dear[dear.duplicated(subset=['Barcode'],keep=False) & dear.Barcode.notnull()]#.to_csv('outputs/duplicated dear barcodes.csv')

dear[dear.duplicated(subset=['Name'],keep=False)]

,ProductCode,Name,Category,Barcode,Price,Status
22,850010806136,1876 Vodka 175L,Vodka,850010806136,23.99,ACTIVE
23,949223659484,1876 Vodka 175L,Vodka,949223659484,21.99,DEPRECATED
2578,08092510009,El Jolgorio Sierrudo 750ml,Mezcal,08092510009,199.99,ACTIVE
2579,860004996323,El Jolgorio Sierrudo 750ml,Mezcal,860004996323,179.99,ACTIVE
2767,875734000547,Ferraton Crozes Hermitage 750ml,Red,875734000547,21.99,DEPRECATED
2768,875734000554,Ferraton Crozes Hermitage 750ml,White,875734000554,19.99,DEPRECATED
3089,1234500000089,Gift Bags,Bag,1234500000089,2.00,ACTIVE
3090,876718027864,Gift Bags,Bag,NaN,2.00,DEPRECATED
3508,085000429266,High Noon Pineapple 4/12oz,RTD,085000429266,10.99,ACTIVE
3509,85000429266,High Noon Pineapple 4/12oz,RTD,85000429266,10.99,DEPRECATED


## Let's start verifying Square items

In [9]:
#or square['Enabled 2'] != 'Y' or square['Enabled 3'] != 'Y' or square['Enabled Warehouse'] != 'Y'
square[(square['Enabled 1'] != 'Y') | (square['Enabled 2'] != 'Y') | (square['Enabled 3'] != 'Y') | (square['Enabled Warehouse'] != 'Y')].to_csv('outputs/square bad location settings.csv')
#.to_csv('outputs/fuckingaroundbadbarcodes.csv')

## Let's do the join

In [10]:
joined = pd.merge(
    dear,
    square,
    how="outer",
    left_on='ProductCode',
    right_on='SKU')#.to_csv('outputs/dearsquareouterjoin.csv')

In [13]:
joined.to_csv('outputs/dearsquareouterjoin.csv')